In [ ]:
import sys
sys.path.append('../')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import torch
import h5py

from utils.datasets import GeneralDataset
from models.DVBCDModel import DVBCDModel

print(torch.__version__)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)

USE_GPU = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if USE_GPU and torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('using device:', DEVICE)

In [ ]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
MODEL_NAME = "0pe_224ps_0.1theta_0.05sigmaRui_60000nsamples"
LOAD_MODEL_PATH = MAIN_PATH + f"weights/{MODEL_NAME}/"
DATASET_PATH = "/work/Camelyon17/work/DECONVOLUCIONES/Original/"
SAVE_PATH = "/work/work_fran/Deep_Var_BCD/results/deconvolutions/" + f"BCDNET_{MODEL_NAME}/"

In [ ]:
dataset = GeneralDataset(DATASET_PATH, patch_size=224, n_samples=100)

In [ ]:
model = DVBCDModel(device=DEVICE)
model.load(LOAD_MODEL_PATH + "best.pt", remove_module=False)

In [ ]:
for idx in range(len(dataset)):
    Y, Y_OD = dataset[idx]
    file = dataset.image_files[idx]
    new_file = file.split(DATASET_PATH)[1]
    new_file = new_file.split('.')[0] + ".Results.mat"
    new_file = SAVE_PATH + new_file
    print(new_file)
    Y_OD = Y_OD.unsqueeze(0)
    Y_OD = Y_OD.to(DEVICE)
    out_Mnet_mean, _, out_Cnet, _ = model.forward(Y_OD)
    out_Mnet_mean = out_Mnet_mean.cpu().detach().numpy()
    out_Mnet_mean = out_Mnet_mean.squeeze(0)
    out_Cnet = out_Cnet.cpu().detach().numpy()
    out_Cnet = out_Cnet.squeeze(0)

    if not os.path.exists(os.path.dirname(new_file)):
        os.makedirs(os.path.dirname(new_file))

    h5f = h5py.File(new_file, 'w')
    h5f.create_dataset('M', data=out_Mnet_mean)
    h5f.create_dataset('stains', data=out_Cnet)
    h5f.close()
    break